# Train Sentiment Analysis Model

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant

2023-11-16 15:10:35.937373: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [157]:
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

import pandas as pd
from sklearn.model_selection import train_test_split

In [158]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

from datasets import Dataset, DatasetDict

In [160]:
train = pd.read_csv("twitter_training.csv")
validation = pd.read_csv("twitter_validation.csv")

In [161]:
train.columns = ["id", "palavra-chave", "sentimento", "tweet"]

In [162]:
train.dropna(inplace=True)

In [163]:
data = train[["tweet", "sentimento"]]

In [164]:
mapping_dict = {"Positive": 0, "Neutral": 1, "Negative": 2, "Irrelevant": 1}

# Use the map function to replace string values with integers
data["sentimento"] = data["sentimento"].map(mapping_dict)

data["sentimento"] = data["sentimento"].astype(int)
data["sentimento"]

data_train, data_test = train_test_split(data, test_size=0.1)

data_train.head(10)

<ipython-input-164-0e72ab51bd42>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentimento'] = data['sentimento'].map(mapping_dict)
<ipython-input-164-0e72ab51bd42>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentimento'] = data['sentimento'].astype(int)


tweet  sentimento
6416   People are so wicked in this life they will kn...           0
36396  IGP 26 times! This time it is @ superhys and @...           1
42946                 <unk> shock to desi mom's this eid           1
19969  So I was doing the Lower Blackrock Spire with ...           1
14806                    Yup og is my new word a f nigma           1
49538     I never spend my money on fifa points anyways.           2
8596   It's weird how Overwatch doesn't enable anymor...           2
13793  Me: hasn’t watched a single game this season ....           1
39242  This is bullshit, says @PlayHearthstone <unk> ...           2
20876  @Wobblespurt and I had an amazing encounter. I...           0

In [166]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [167]:
def tokenize_function(example):
    return tokenizer(example["tweet"], truncation=True)

In [168]:
train_dataset = Dataset.from_pandas(data_train)
test_dataset = Dataset.from_pandas(data_test)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

In [169]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/66595 [00:00<?, ? examples/s]

Map:   0%|          | 0/7400 [00:00<?, ? examples/s]

In [170]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [171]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["sentimento"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["sentimento"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [174]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [179]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=5)

Epoch 1/5
8325/8325 [==============================] - 1414s 164ms/step - loss: 0.6042 - accuracy: 0.7451 - val_loss: 0.3504 - val_accuracy: 0.8689
Epoch 2/5
8325/8325 [==============================] - 1320s 159ms/step - loss: 0.2387 - accuracy: 0.9104 - val_loss: 0.2092 - val_accuracy: 0.9253
Epoch 3/5
8325/8325 [==============================] - 1310s 157ms/step - loss: 0.1230 - accuracy: 0.9524 - val_loss: 0.2296 - val_accuracy: 0.9288
Epoch 4/5
8325/8325 [==============================] - 1306s 157ms/step - loss: 0.0892 - accuracy: 0.9633 - val_loss: 0.2171 - val_accuracy: 0.9347
Epoch 5/5
8325/8325 [==============================] - 1338s 161ms/step - loss: 0.0733 - accuracy: 0.9697 - val_loss: 0.1888 - val_accuracy: 0.9443


In [180]:
from huggingface_hub import notebook_login

notebook_login()

In [181]:
model.push_to_hub("my-awesome-bert-model-sentiment-analysis")
tokenizer.push_to_hub("my-awesome-bert-model-sentiment-analysis")

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Guspfc/my-awesome-bert-model-sentiment-analysis/commit/a80034e747e037a97821baff6a1cabd5fc3f83a0', commit_message='Upload tokenizer', commit_description='', oid='a80034e747e037a97821baff6a1cabd5fc3f83a0', pr_url=None, pr_revision=None, pr_num=None)